# 0. Env

In [ ]:
import os
import glob
import json

import numpy as np

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

import faiss
import chromadb

In [ ]:
# 데이터를 저장할 경로
data_home = "data"

# 1. Wiki 데이터 준비

In [ ]:
# 데이터 폴더 생성
os.makedirs(data_home, exist_ok=True)

In [ ]:
# 최신 wiki dump 다운로드
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2 \
    -O ./{data_home}/kowiki-latest-pages-articles.xml.bz2

In [ ]:
# wiki dump 파일 전처리
# 시간이 오래 걸립니다.
!/home/kysman/venvs/gen-ai/bin/python \
    -m wikiextractor.WikiExtractor \
    --json \
    --out ./{data_home}/kowiki \
    ./{data_home}/kowiki-latest-pages-articles.xml.bz2

In [ ]:
# 파일 목록 확인
fn_list = glob.glob(f'{data_home}/kowiki/*/*')
fn_list.sort()
fn_list[:10]

In [ ]:
# wiki 1개 페이지만 확인
with open(fn_list[0]) as f:
    for line in f:
        page = json.loads(line)
        print(page)
        print(page['title'])
        print(page['text'])
        break

In [ ]:
def make_chunk(text, n_word=128):
    # line 단위로 단어수 계산
    line_list = []
    total = 0
    for line in text.split('\n'):
        total += len(line.split())
        line_list.append((total, line))
    # n_word 단위로 분할
    chunk_list = []
    chunk_total, chunk_index = 0, 0
    for i, (total, line) in enumerate(line_list):
        if total - chunk_total >= n_word:
            chunk = [line for total, line in line_list[chunk_index:i+1]]
            chunk_list.append('\n'.join(chunk))
            chunk_index = i + 1
            chunk_total = total
    # 마지막 line 추가 (n_word 보다 작은 경우 이전라인 포함)
    if total > chunk_total:
        if total - chunk_total < n_word and chunk_index > 1:
            chunk_index -= 1
        chunk = [line for total, line in line_list[chunk_index:]]
        chunk_list.append('\n'.join(chunk))
    return chunk_list

In [ ]:
# 기능 확인을 위해서 문서를 chunk 단위로 분할해서 row_list에 저장
# 이유는 각 페이지의 문서의 길이가 너무 다르기 때문에 적당한 길이로 페이지를 분할
row_list = []
for fn in fn_list[:100]:  # 100개 파일만 사용 (1만개 위키 페이지)
    with open(fn) as f:
        for line in f:
            data = json.loads(line)
            chunk_list = make_chunk(data['text'])
            for i, chunk in enumerate(chunk_list):
                title = data['title']
                row = {
                    'id': data['id'],
                    'chunk_id': str(i + 1),
                    'chunk': f"{title}\n{chunk}"
                }
                print(row)
                row_list.append(row)
len(row_list)

In [ ]:
# chunk를 저장합니다.
with open("data/chunk_db.json", "w") as f:
    for row in row_list:
        f.write(json.dumps(row, ensure_ascii=False))
        f.write("\n")

In [ ]:
# chunk 내용을 확인합니다.
!head -n 5 ./data/chunk_db.json

# 2. 임베딩 만들기 (Sentence Bert)

In [ ]:
# SentenceBERT 모델 생성
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [ ]:
# full chunks (실습 용)
full_chunks = []
with open("data/chunk_db.json") as f:
    for line in f:
        row = json.loads(line)
        full_chunks.append(row['chunk'])
len(full_chunks)

In [ ]:
# chunk embedding 생성
# 시간 오래 걸림
chunk_embeddings = model.encode(full_chunks, normalize_embeddings=True)
chunk_embeddings.shape

In [ ]:
# chunk norm 확인
np.linalg.norm(chunk_embeddings, axis=1)

In [ ]:
# 질문
query = "지미 카터가 졸업한 대학은 어디야?"
# query embedding
query_embedding = model.encode(query, normalize_embeddings=True)
query_embedding.shape

In [ ]:
# query norm 확인
np.linalg.norm(query_embedding, axis=0)

# 3. 직접 계산

In [ ]:
# cosine sim
scores = np.dot(chunk_embeddings, query_embedding)
scores.shape

In [ ]:
# 유사도 순 정랼
ranks = np.argsort(-scores)
ranks

In [ ]:
# 결과 score 확인 (top-10)
for i in ranks[:10]:
    print("=" * 10, scores[i], "=" * 10)
    print(full_chunks[i])
    print()

# 4. FAISS

In [ ]:
# dimension of embedding
d = chunk_embeddings.shape[1]
d

## 4.1. L2 distance

In [ ]:
# L2 Index 생성 (L2 Distance)
index = faiss.IndexFlatL2(d)
type(index)

In [ ]:
# check index available
index.is_trained

In [ ]:
# add chunk embedding
index.add(chunk_embeddings)

In [ ]:
# check total embedding number
index.ntotal

In [ ]:
query_embeddings = query_embedding.reshape(1, -1)
query_embeddings.shape

In [ ]:
%%time
D, I = index.search(query_embeddings, 10)  # search
print(I)

In [ ]:
# 결과 확인
for i in I[0]:
    print(i)
    print(full_chunks[i])
    print('=' * 30)

In [ ]:
# index 저장하기
faissindex_file = "data/faiss_flat_l2.index"
faiss.write_index(index, faissindex_file)

In [ ]:
# index 읽어오기
load_index = faiss.read_index(faissindex_file)
type(load_index), load_index.ntotal

## 4.2. Inner Product Query (연습 문제)
- 위 코드와 비슷하게 tutorial 코드를 완성하세요. (모든 동작이 동일합니다.)

In [ ]:
# Inner Product Index 생성
index = faiss.IndexFlatIP(d)
type(index)

# 5. Chroma DB

In [ ]:
# in memory db
# https://docs.trychroma.com/reference/Client#client
# client = chromadb.Client()

In [ ]:
# 데이터를 파일에 저장 (sqlite)
# https://docs.trychroma.com/reference/Client#persistentclient
client = chromadb.PersistentClient(path="data/chroma.db")

## 5.1. L2 distance Query

In [ ]:
try:
    # 자동차 메뉴얼 l2 distance collection
    car_l2 = client.create_collection(name="car_l2")
    print('create:', 'car_l2')
except:
    car_l2 = client.get_collection(name="car_l2")
    print('exists:', 'car_l2')
car_l2

In [ ]:
# np.array to list
chunk_embedding_list = [embedding.tolist() for embedding in chunk_embeddings]
# dimension of embedding
len(chunk_embedding_list)

In [ ]:
# 문서 목록
metadatas = [{'text': t, 'source': 'kowiki'} for t in full_chunks]
len(metadatas)

In [ ]:
# ids (string only)
ids = list([str(i) for i in range(len(metadatas))])
len(ids)

In [ ]:
# 데이터 입력
car_l2.add(embeddings=chunk_embedding_list,
             metadatas=metadatas,
             ids=ids)

In [ ]:
# l2 distance query
result = car_l2.query(query_embedding.tolist(),
                      n_results=10)
print(result)

In [ ]:
for i in range(len(result['ids'][0])):
    print(result['metadatas'][0][i]['text'])
    print('=' * 30)

## 5.2. Cosine sim Query (연습 문제)
- 위 코드와 비슷하게 tutorial 코드를 완성하세요. (모든 동작이 동일합니다.)

In [ ]:
try:
    # 자동차 메뉴얼 l2 distance collection
    car_cos = client.create_collection(name="car_cos", metadata={"hnsw:space": "cosine"})
    print('create:', 'car_cos')
except:
    car_cos = client.get_collection(name="car_cos")
    print('exists:', 'car_cos')
car_cos